## Analyze whether SNWD varies more from year to year or from place to place.

In [29]:
sc.stop()

In [1]:
import pandas as pd
import numpy as np
import urllib
import math

In [2]:
import findspark
findspark.init()

from pyspark import SparkContext
#sc.stop()
sc = SparkContext(master="local[3]",pyFiles=['lib/numpy_pack.py','lib/spark_PCA.py','lib/computeStats.py'])

from pyspark import SparkContext
from pyspark.sql import *
sqlContext = SQLContext(sc)

In [3]:
import sys
sys.path.append('./lib')

import numpy as np
from numpy_pack import packArray,unpackArray
from spark_PCA import computeCov
from computeStats import computeOverAllDist, STAT_Descriptions

In [4]:
### Read the data frame from pickle file

data_dir='../../Data/Weather'
file_index='BSBSSSBB'
meas='SNWD'

from pickle import load

#read statistics
filename=data_dir+'/STAT_%s.pickle'%file_index
STAT,STAT_Descriptions = load(open(filename,'rb'))
print('keys from STAT=',STAT.keys())

#!ls -ld $data_dir/*.parquet

#read data
filename=data_dir+'/decon_%s_%s.parquet'%(file_index,meas)

df=sqlContext.read.parquet(filename)
print(df.count())
df.show(2)

('keys from STAT=', ['TMIN', 'TOBS', 'TMAX', 'SNOW', 'SNWD', 'PRCP'])
44
+-------------------+-----------------+------------------+--------------------+---------+--------+--------+---------+-----------+------------------+-----------------+------------------+------------------+------------------+-----------+---------+------+--------------------+------+
|            coeff_1|          coeff_2|           coeff_3|             coeff_4|elevation|   label|latitude|longitude|measurement|             res_1|            res_2|             res_3|             res_4|          res_mean|    station|total_var|undefs|              vector|  year|
+-------------------+-----------------+------------------+--------------------+---------+--------+--------+---------+-----------+------------------+-----------------+------------------+------------------+------------------+-----------+---------+------+--------------------+------+
|-0.4664808631291546|5.489328404086392|-20.26409268620201|-0.04799803786458...|     

In [5]:
print df.columns

['coeff_1', 'coeff_2', 'coeff_3', 'coeff_4', 'elevation', 'label', 'latitude', 'longitude', 'measurement', 'res_1', 'res_2', 'res_3', 'res_4', 'res_mean', 'station', 'total_var', 'undefs', 'vector', 'year']


In [20]:
#extract longitude and latitude for each station
feature='coeff_4'
sqlContext.registerDataFrameAsTable(df,'weather')
#Features=', '.join(['coeff_1', 'coeff_2', 'coeff_3', 'elevation', 'latitude', 'longitude',\
#          'res_1', 'res_2', 'res_3', 'res_mean', 'year'])
Features='station, year, coeff_4'
Query="SELECT %s FROM weather"%Features
print(Query)
pdf = sqlContext.sql(Query).toPandas()
pdf.head()

SELECT station, year, coeff_4 FROM weather


,station,year,coeff_4
0,USC00090670,2010.0,-0.047998
1,USC00097171,2010.0,-0.047998
2,USC00094204,2010.0,-0.047998
3,USC00085275,1989.0,-0.158081
4,USW00013870,1989.0,-0.158081


In [21]:
year_station_table=pdf.pivot(index='year', columns='station', values='coeff_4')
year_station_table.head(10)

station,US1GALN0001,USC00084394,USC00085099,USC00085275,USC00090010,USC00090090,USC00090670,USC00092783,USC00092966,USC00093386,...,USC00095874,USC00096237,USC00097171,USC00097777,USC00098223,USC00098496,USC00098972,USW00013857,USW00013861,USW00013870
year,,,,,,,,,,,,,,,,,,,,,
1958.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.005281,NaN,NaN,NaN,NaN,NaN,0.005281,NaN,NaN
1961.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1967.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.200272,NaN,NaN,-0.612533,NaN,NaN,NaN,NaN,NaN,NaN
1968.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.118532,NaN,NaN,NaN,NaN,NaN,NaN,0.004967,NaN,NaN
1973.0,NaN,-0.200272,NaN,NaN,NaN,NaN,NaN,-0.251806,NaN,NaN,...,0.440640,NaN,NaN,NaN,NaN,-0.919707,NaN,NaN,NaN,-0.097209
1977.0,NaN,NaN,NaN,NaN,NaN,-0.115959,NaN,NaN,-0.104441,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1978.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1979.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.141909,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1980.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.115657,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Estimating the effect of the year vs the effect of the station

To estimate the effect of time vs. location on the first eigenvector coefficient we
compute:

* The average row: `mean-by-station`
* The average column: `mean-by-year`

We then compute the RMS before and after subtracting either  the row or the column vector.

In [22]:
def RMS(Mat):
    return np.sqrt(np.nanmean(Mat**2))

mean_by_year=np.nanmean(year_station_table,axis=1)
mean_by_station=np.nanmean(year_station_table,axis=0)
tbl_minus_year = (year_station_table.transpose()-mean_by_year).transpose()
tbl_minus_station = year_station_table-mean_by_station

print 'total RMS                   = ',RMS(year_station_table)
print 'RMS removing mean-by-station= ',RMS(tbl_minus_station)
print 'RMS removing mean-by-year   = ',RMS(tbl_minus_year)

total RMS                   =  30.588638776
RMS removing mean-by-station=  0.0894169101026
RMS removing mean-by-year   =  21.6407752978


In [9]:
T=year_station_table
print 'initial RMS=',RMS(T)
for i in range(5):
    mean_by_year=np.nanmean(T,axis=1)
    T=(T.transpose()-mean_by_year).transpose()
    print i,'after removing mean by year    =',RMS(T)
    mean_by_station=np.nanmean(T,axis=0)
    T=T-mean_by_station
    print i,'after removing mean by stations=',RMS(T)

initial RMS= 83.7824184311
0 after removing mean by year    = 63.0150971492
0 after removing mean by stations= 52.5407076788
1 after removing mean by year    = 48.633276032
1 after removing mean by stations= 46.4028692706
2 after removing mean by year    = 44.9107264101
2 after removing mean by stations= 43.8036748611
3 after removing mean by year    = 42.9327193274
3 after removing mean by stations= 42.2249267708
4 after removing mean by year    = 41.6386305674
4 after removing mean by stations= 41.1468115032


In [40]:
sc.stop()